In [4]:
import csv
from typing import List, Optional, Tuple

from fastapi import FastAPI, Depends, HTTPException, status, APIRouter, Header
from pydantic import BaseModel, Field, ConfigDict
from sqlalchemy import create_engine, Column, Integer, String, Float, select, func
from sqlalchemy.orm import declarative_base, Session, sessionmaker

# ---------- БД и ORM ----------
DATABASE_URL = "sqlite:///students.db"
engine = create_engine(DATABASE_URL, future=True, echo=False)
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False, future=True)

Base = declarative_base()


class Student(Base):
    __tablename__ = "students"
    id = Column(Integer, primary_key=True, autoincrement=True)
    last_name = Column(String, nullable=False)
    first_name = Column(String, nullable=False)
    faculty = Column(String, nullable=False)
    course = Column(String, nullable=False)
    grade = Column(Float, nullable=False)


class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, autoincrement=True)
    username = Column(String, unique=True, nullable=False)
    password = Column(String, nullable=False)  # в учебных целях без хеширования


Base.metadata.create_all(engine)

# ---------- Pydantic v2 схемы ----------


class StudentBase(BaseModel):
    last_name: str = Field(..., json_schema_extra={"example": "Иванов"})
    first_name: str = Field(..., json_schema_extra={"example": "Иван"})
    faculty: str = Field(..., json_schema_extra={"example": "ФПМИ"})
    course: str = Field(..., json_schema_extra={"example": "Мат. Анализ"})
    grade: float = Field(..., ge=0, le=100, json_schema_extra={"example": 72})


class StudentCreate(StudentBase):
    pass


class StudentUpdate(BaseModel):
    last_name: Optional[str] = Field(None, json_schema_extra={"example": "Петров"})
    first_name: Optional[str] = Field(None, json_schema_extra={"example": "Пётр"})
    faculty: Optional[str] = Field(None, json_schema_extra={"example": "РЭФ"})
    course: Optional[str] = Field(None, json_schema_extra={"example": "Информатика"})
    grade: Optional[float] = Field(None, ge=0, le=100, json_schema_extra={"example": 88})


class StudentOut(StudentBase):
    id: int
    model_config = ConfigDict(from_attributes=True)


class UserBase(BaseModel):
    username: str = Field(..., json_schema_extra={"example": "user1"})


class UserCreate(UserBase):
    password: str = Field(..., min_length=4, json_schema_extra={"example": "secret"})


class UserLogin(UserBase):
    password: str = Field(..., min_length=4, json_schema_extra={"example": "secret"})


class UserOut(UserBase):
    id: int
    model_config = ConfigDict(from_attributes=True)


class AvgOut(BaseModel):
    faculty: str
    average_grade: float


# ---------- DI: сессия и текущий пользователь ----------


def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


def get_current_user(
    x_user_id: int = Header(..., alias="X-User-Id"),
    db: Session = Depends(get_db),
) -> User:
    user = db.get(User, x_user_id)
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Неверный X-User-Id или пользователь не существует",
        )
    return user


# ---------- Репозиторий студентов ----------


class StudentRepo:
    def __init__(self, db: Session):
        self.db = db

    # Create
    def create(self, data: StudentCreate) -> Student:
        obj = Student(**data.model_dump())
        self.db.add(obj)
        self.db.commit()
        self.db.refresh(obj)
        return obj

    # Read
    def get(self, student_id: int) -> Student:
        obj = self.db.get(Student, student_id)
        if not obj:
            raise HTTPException(status_code=404, detail="Студент не найден")
        return obj

    def list(self, limit: int = 100, offset: int = 0) -> List[Student]:
        stmt = select(Student).offset(offset).limit(limit)
        return list(self.db.execute(stmt).scalars())

    # Update (partial)
    def update(self, student_id: int, data: StudentUpdate) -> Student:
        obj = self.get(student_id)
        for k, v in data.model_dump(exclude_unset=True).items():
            setattr(obj, k, v)
        self.db.commit()
        self.db.refresh(obj)
        return obj

    # Delete
    def delete(self, student_id: int) -> None:
        obj = self.get(student_id)
        self.db.delete(obj)
        self.db.commit()

    # ---- Запросы из задания ----
    def by_faculty(self, faculty: str) -> List[Student]:
        stmt = (
            select(Student)
            .where(Student.faculty == faculty)
            .order_by(Student.last_name, Student.first_name)
        )
        return list(self.db.execute(stmt).scalars())

    def unique_courses(self) -> List[str]:
        stmt = select(func.distinct(Student.course)).order_by(Student.course)
        return [row[0] for row in self.db.execute(stmt).all()]

    def avg_grade_by_faculty(self) -> List[Tuple[str, float]]:
        stmt = (
            select(Student.faculty, func.avg(Student.grade))
            .group_by(Student.faculty)
            .order_by(Student.faculty)
        )
        return [(fac, float(avg)) for fac, avg in self.db.execute(stmt).all()]

    def below_grade_in_course(self, course: str, threshold: float = 30.0) -> List[Student]:
        stmt = (
            select(Student)
            .where((Student.course == course) & (Student.grade < threshold))
            .order_by(Student.grade.asc())
        )
        return list(self.db.execute(stmt).scalars())

    # Утилита: загрузка CSV
    def load_csv(self, csv_path: str) -> int:
        added = 0
        with open(csv_path, "r", encoding="utf-8-sig", newline="") as f:
            reader = csv.DictReader(f)
            required = ["Фамилия", "Имя", "Факультет", "Курс", "Оценка"]
            missing = [c for c in required if c not in (reader.fieldnames or [])]
            if missing:
                raise HTTPException(
                    status_code=400, detail=f"В CSV нет колонок: {missing}"
                )
            for row in reader:
                self.db.add(
                    Student(
                        last_name=row["Фамилия"].strip(),
                        first_name=row["Имя"].strip(),
                        faculty=row["Факультет"].strip(),
                        course=row["Курс"].strip(),
                        grade=float(row["Оценка"]),
                    )
                )
                added += 1
            self.db.commit()
        return added


# ---------- FastAPI и роутеры ----------

app = FastAPI(title="Students CRUD (Pydantic v2 + Auth)")

auth_router = APIRouter(prefix="/auth", tags=["auth"])


# ----- /auth эндпоинты -----


@auth_router.post(
    "/register", response_model=UserOut, status_code=status.HTTP_201_CREATED
)
def register_user(payload: UserCreate, db: Session = Depends(get_db)):
    existing = (
        db.execute(select(User).where(User.username == payload.username))
        .scalar_one_or_none()
    )
    if existing:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Пользователь с таким username уже существует",
        )
    user = User(username=payload.username, password=payload.password)
    db.add(user)
    db.commit()
    db.refresh(user)
    return user


@auth_router.post("/login")
def login_user(payload: UserLogin, db: Session = Depends(get_db)):
    user = (
        db.execute(select(User).where(User.username == payload.username))
        .scalar_one_or_none()
    )
    if not user or user.password != payload.password:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Неверный логин или пароль",
        )
    # базовый вариант: возвращаем идентификатор, который клиент кладёт в X-User-Id
    return {"user_id": user.id}


@auth_router.post("/logout")
def logout_user(current_user: User = Depends(get_current_user)):
    # клиент просто перестаёт слать X-User-Id
    return {"detail": f"Пользователь {current_user.username} разлогинен"}


app.include_router(auth_router)

# ----- CRUD по студентам -----


@app.post("/students", response_model=StudentOut, status_code=status.HTTP_201_CREATED)
def create_student(
    payload: StudentCreate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).create(payload)


@app.get("/students/{student_id}", response_model=StudentOut)
def get_student(
    student_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).get(student_id)


@app.get("/students", response_model=List[StudentOut])
def list_students(
    limit: int = 100,
    offset: int = 0,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).list(limit=limit, offset=offset)


@app.put("/students/{student_id}", response_model=StudentOut)
def update_student(
    student_id: int,
    payload: StudentUpdate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).update(student_id, payload)


@app.delete("/students/{student_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_student(
    student_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    StudentRepo(db).delete(student_id)
    return


# ----- Выборки из задания -----


@app.get("/students/by-faculty/{faculty}", response_model=List[StudentOut])
def students_by_faculty(
    faculty: str,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).by_faculty(faculty)


@app.get("/courses/unique", response_model=List[str])
def get_unique_courses(
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).unique_courses()


@app.get("/faculties/avg-grade", response_model=List[AvgOut])
def avg_grade_by_faculty(
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    data = StudentRepo(db).avg_grade_by_faculty()
    return [{"faculty": fac, "average_grade": avg} for fac, avg in data]


@app.get("/students/by-course/below", response_model=List[StudentOut])
def below_threshold(
    course: str,
    threshold: float = 30.0,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    return StudentRepo(db).below_grade_in_course(course, threshold)


# Утилита: загрузить CSV в БД (по желанию)
@app.post("/load-csv")
def load_csv(
    path: str,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    added = StudentRepo(db).load_csv(path)
    return {"added": added}
